## Hey,It was really great to be part of this competition. Learnt a lot of new stuff.

1. I have reffred a lot from existing public notebooks. Lovely Thanks for that.**

2. Used CatBoost and 2 LGBM models, which is trained through 5 GroupKFold by time_id. 

3. I have used very few feature, only about 30-40.

4. Optuna was my library for Hyperparameter tuning. 

5. Also used Kmeans Algo on columns which cor-related highly with target column.

7. Had other stuff going on, I was able to put very little effort on this competition(Some where about 4-5 days). I wish I had more time to be spent on this competition, I could have learnt more

8. **With this Notebook I was promoted to notebook expert. Thank You for your support.**

**All the Best**

# Imports

In [ ]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np 
import pandas as pd
import glob
import os
import gc

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.cluster import KMeans
import lightgbm as lgb

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns

path_data = '../input/optiver-realized-volatility-prediction/'

import warnings
warnings.filterwarnings('ignore')

# Dataset Pre-load

In [ ]:
%%time
StockID = True
if StockID:
    df_training = pd.read_pickle('../input/optiver-dataset-version-2/training.pkl')
else:
    df_training = pd.read_pickle('../input/optiver-dataset-version-2/training_no_stock_id.pkl')

In [ ]:
df_training = df_training.drop(['wap2_mean', 'wap2_std'], axis = 1)

# Kmeans

In [ ]:
%%time
cols_kmeans = ['log_return1_realized_volatility', 'log_return1_realized_absvar',
    'log_return1_realized_quarticity','log_return1_realized_tripower_quarticity',
    'log_return1_realized_quadpower_quarticity', 'log_return1_realized_1',
    'log_return1_realized_2']
clustering = False
if clustering:
    SSE = []
    for cluster in range(1,11):
        print(f'Done Cluster: {cluster}')
        kmeans = KMeans(n_init = 8, n_clusters = cluster, init='k-means++')
        kmeans.fit(df_training[cols])
        SSE.append(kmeans.inertia_)
    # Elbow Method
    frame = pd.DataFrame({'Cluster':range(1,11), 'SSE':SSE})
    plt.figure(figsize=(12,6))
    plt.plot(frame['Cluster'], frame['SSE'], marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')

In [ ]:
%%time 

KMEAN = True
if KMEAN:
    kmeans = KMeans(n_init = 20, n_clusters = 8, init='k-means++')

    kmeans.fit(df_training[cols_kmeans].fillna(0))
        
    df_training['kmeans'] = kmeans.labels_

In [ ]:
df_training['kmeans']

In [ ]:
#%%time
#df_testing = pd.read_pickle('../input/optiver-dataset-version-1/testing.pkl')

# Preprocessing && Test Data 

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def realized_absvar(series):
    return np.sqrt(np.pi/(2*series.count()))*np.sum(np.abs(series))

# Calculate integrated quarticity
def realized_quarticity(series):
    return (series.count()/3)*np.sum(series**4)

def calc_depth(df):
    depth = df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1'] + df['bid_price2'] * df[
               'bid_size2'] + df['ask_price2'] * df['ask_size2']
    return depth

def realized_3(series):
    from scipy.special import gamma
    numerator = (gamma(1/2)**3)*np.sum((abs(series)**(4/3)).rolling(window=3).apply(np.prod))
    denominator = 8 * (gamma(7/6)**3)*np.sum(series**2)
    return np.sqrt(numerator/denominator)

def realized_2(series):
    return np.sqrt(((np.pi**2)*np.sum(abs(series.rolling(window=4).apply(np.product, raw=True))))/(8*np.sum(series**2)))

def realized_1(series):
    return np.sqrt(np.sum(series**4)/(6*np.sum(series**2)))

def realized_tripower_quarticity(series):
    from scipy.special import gamma
    series = series ** (4/3)
    series = abs(series).rolling(window=3).apply(np.prod, raw=True)
    return series.shape[0]*0.25*((gamma(1/2)**3)/(gamma(7/6)**3))*np.sum(series)

def realized_quadpower_quarticity(series):
    series = abs(series.rolling(window=4).apply(np.product, raw=True))
    return (np.sum(series) * series.shape[0] * (np.pi**2))/4

def processing_book_data(file_path):
    
    df = pd.read_parquet(file_path)
    # Wap 1
    df['wap1'] = calc_wap1(df) 
    df['log_return1'] = df.groupby('time_id')['wap1'].apply(log_return)
    # Wap 2 
    df['wap2'] = calc_wap2(df)
    df['log_return2'] = df.groupby('time_id')['wap2'].apply(log_return)
    
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1'])/2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    df['depth'] = calc_depth(df)
    
    create_feature_dict = {
        'log_return1':[realized_volatility,realized_absvar,realized_quarticity,realized_tripower_quarticity,realized_quadpower_quarticity,realized_1,realized_2],
        'log_return2':[realized_volatility,realized_absvar,realized_quarticity,realized_tripower_quarticity,realized_quadpower_quarticity,realized_1,realized_2],
        'wap_balance':[np.mean],
        #'price_spread':[np.mean],
        'bid_spread':[np.mean,np.sum],
        'ask_spread':[np.mean, np.max],
        'volume_imbalance':[np.mean],
        #'total_volume':[np.mean],
        'wap1':[np.mean,np.std],
        #'wap2':[np.mean,np.std]
    }
    create_feature_dict_time = {
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility]
    }
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    # 300 and 400 
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    # 600
    df_feature = pd.DataFrame(df.groupby(['time_id']).agg(create_feature_dict)).reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]
    # merge
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature.drop(['time_id__400', 'time_id__300'], axis = 1, inplace = True)
    
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['time_id_'],axis=1)
    
    return df_feature

def processing_trade_data(file_path):
    
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    aggregate_dictionary = {
        'log_return':[realized_volatility],
    }
    
    create_feature_dict_time = {
        'log_return':[realized_volatility],
    }
    
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    df_feature = df.groupby('time_id').agg(aggregate_dictionary)
    df_feature = df_feature.reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    
    df_feature.drop(['time_id__400', 'time_id__300'], axis = 1, inplace = True)
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)

    return df_feature

def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility',  
                'log_return1_realized_volatility_400',
                'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300',
                'log_return2_realized_volatility_300',
                'trade_log_return_realized_volatility', 
                'trade_log_return_realized_volatility_400',
                'trade_log_return_realized_volatility_300']

    valo_cols1 = ['log_return1_realized_volatility','trade_log_return_realized_volatility_400']
    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'max', 'min' ]).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    feature_column_needed = [ 'stock_id__stock' ,'log_return1_realized_volatility_mean_stock', 
       'log_return1_realized_volatility_max_stock',
       'log_return1_realized_volatility_min_stock',
       'log_return1_realized_volatility_400_max_stock',
       'log_return1_realized_volatility_400_min_stock',
       'log_return2_realized_volatility_400_min_stock',
       'log_return1_realized_volatility_300_min_stock',
       'log_return2_realized_volatility_300_min_stock',
       'trade_log_return_realized_volatility_mean_stock',
       'trade_log_return_realized_volatility_max_stock',
       'trade_log_return_realized_volatility_400_max_stock',
       'trade_log_return_realized_volatility_400_min_stock',
       'trade_log_return_realized_volatility_300_min_stock',]
    
    df_stock_id_column = pd.DataFrame(df_stock_id, columns = feature_column_needed)
    
    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[valo_cols1].agg(['mean', 'min' ]).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    feature_column_needed = ['time_id__time',
        'log_return1_realized_volatility_mean_time',
       'trade_log_return_realized_volatility_400_min_time']
    
    df_time_id_column = pd.DataFrame(df_time_id, columns= feature_column_needed)
    
    
    # Merge with original dataframe
    df = df.merge(df_stock_id_column, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id_column, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    
    return df

def read_train_test():
    #train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    # Create a key to merge with book and trade data
    #train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    #print(f'Our training set has {train.shape[0]} rows')
    return test

def preprocessor(list_stock_ids, is_train = True):
    from joblib import Parallel, delayed # parallel computing to save time
    df = pd.DataFrame()
    
    def for_joblib(stock_id):
        if is_train:
            file_path_book = path_data + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = path_data + "trade_train.parquet/stock_id=" + str(stock_id)
        else:
            file_path_book = path_data + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = path_data + "trade_test.parquet/stock_id=" + str(stock_id)
            
        df_tmp = pd.merge(processing_book_data(file_path_book),processing_trade_data(file_path_trade),on='row_id',how='left')
     
        return pd.concat([df,df_tmp])
    
    df = Parallel(n_jobs=-1, verbose=1)(
        delayed(for_joblib)(stock_id) for stock_id in list_stock_ids
        )

    df =  pd.concat(df,ignore_index = True)
    return df

In [ ]:
%%time
test = read_train_test()

test_ids = test.stock_id.unique()

df_test = preprocessor(list_stock_ids= test_ids, is_train = False)
df_testing = test.merge(df_test, on = ['row_id'], how = 'left')
df_testing['kmeans'] = kmeans.predict(df_testing[cols_kmeans].fillna(0))
if StockID:
    df_testing = get_time_stock(df_testing)
    df_testing.drop(['trade_log_return_realized_volatility_400', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_300', 'trade_log_return_realized_volatility_300', 'log_return2_realized_volatility_400','log_return1_realized_volatility_400'], axis = 1, inplace = True)
else:
    pass

In [ ]:
df_testing.shape

# Correlation 

In [ ]:
#import seaborn as sns
#plt.figure(figsize=(30,30))
#corr = df_training.corr()
#sns.heatmap(corr,cmap='Blues',linewidth=0.5,annot=True)

# Model && Tuning 

In [ ]:
import optuna 
from sklearn.model_selection import GroupKFold
import lightgbm as lgb

def rmspe(y_true, y_pred):
    return (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def rmspe_xgb(pred, dtrain):
    y = dtrain.get_label()
    return 'rmspe', rmspe(y, pred)

def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

# LGBM Tuning

In [ ]:
num_folds = 5
group_fold = GroupKFold( n_splits = num_folds )

features = [col for col in df_training.columns if col not in { "target", "row_id"}]
X = df_training[features]
y = df_training['target']

def objective(trial):

    # Optuna suggest params
    seed = 1111
    params = {
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.2),        
        'lambda_l1': trial.suggest_float('lambda_l1', 0.1, 1),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.1, 1),
        'num_leaves': trial.suggest_int('num_leaves', 400, 800),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.50, 1),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.50, 1),
        'bagging_freq': trial.suggest_int('bagging_freq',1, 2),
        'min_data_in_leaf':trial.suggest_int('min_data_in_leaf',400,700),
        'max_depth': trial.suggest_int('max_depth', 6 , 13),
        'seed': seed,
        'objective': 'rmse',
        'boosting': 'gbdt',
        'verbosity': -1,
        'n_jobs': -1,
        #'device': 'gpu','gpu_platform_id': 0,
        #'gpu_device_id': 0
    }  

    
    rmspe_list = []

    for fold, (trn_ind, val_ind) in enumerate(group_fold.split(X, y, groups = X['time_id'])):
        print(f'Training fold {fold + 1}')
        x_train, x_val = X.iloc[trn_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train, y_train, weight = train_weights)
        val_dataset = lgb.Dataset(x_val, y_val, weight = val_weights)
        model = lgb.train(params = params,
                          num_boost_round=5000,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          verbose_eval = 250,
                          early_stopping_rounds=20,
                          feval = feval_rmspe)
        
        #preds = model.predict(d_val)
        preds = model.predict(x_val[features])
        score = rmspe(y_val, preds)
        rmspe_list.append(score)
    
    print(f'Trial done: rmspe values on folds: {np.mean(rmspe_list)}')
    return np.mean(rmspe_list)

In [ ]:
n_trials = 10

FIT_LGB = False

if FIT_LGB:
    study = optuna.create_study(direction="minimize",study_name = 'LGB')
    study.optimize(objective)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

# CatBoost Tuning 

In [ ]:
from catboost import CatBoostRegressor, Pool
num_folds = 5
group_fold = GroupKFold( n_splits = num_folds )

features = [col for col in df_training.columns if col not in { "target", "row_id"}]
X = df_training[features]
y = df_training['target']
def objective(trial):

    params = {
        #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1),
        "depth": trial.suggest_int("depth", 6, 11),
        "learning_rate": trial.suggest_uniform("learning_rate",0.01,0.2),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg",0.5,1),
        "random_seed": 42,
        "iterations": 5000,
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Lossguide", "Depthwise"]),
        "bootstrap_type":"Bayesian", 
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 5),
        "loss_function" : 'RMSE',
        "eval_metric":"RMSE",
        "has_time": True
        #"task_type":'GPU'
    }
    
    if params["grow_policy"] == "Depthwise" or params["grow_policy"] == "Lossguide":
        params["min_data_in_leaf"] = trial.suggest_int("min_data_in_leaf", 400, 700)
    if params["grow_policy"] == "Lossguide":
        params["max_leaves"] = trial.suggest_int("max_leaves", 10, 20)
    
    #
    rmspe_list = []

    for fold, (trn_ind, val_ind) in enumerate(group_fold.split(X, y, groups = X['time_id'])):
        print(f'Training fold {fold + 1}')
        x_train, x_val = X.iloc[trn_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        
        train_pool = Pool(x_train, y_train,weight=train_weights)
        val_pool = Pool(x_val, y_val,weight=val_weights)
        
        model = CatBoostRegressor(**params)
        
        model.fit(train_pool,
                  use_best_model = True,
                  eval_set = val_pool, 
                  verbose = 250,
                  early_stopping_rounds=10,
                  plot =True)

        preds = model.predict(x_val[features])
        score = rmspe(y_val, preds)
        rmspe_list.append(score)
    
    print(f'Trial done: rmspe values on folds: {np.mean(rmspe_list)}')
    return np.mean(rmspe_list)

In [ ]:
n_trials = 10

FIT_CAT = False

if FIT_CAT:
    study = optuna.create_study(direction="minimize",study_name = 'CAT')
    study.optimize(objective, n_trials=n_trials)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

# CAT Boost Training

In [ ]:
cat_params = {'depth': 6, 'learning_rate': 0.08814388269284729, 'l2_leaf_reg': 0.863393460851281, 
              'grow_policy': 'Lossguide', 'bagging_temperature': 0.9196224418501292, 'min_data_in_leaf': 443, 
              'max_leaves': 20,"random_seed": 42,"iterations": 5000,
              "bootstrap_type":"Bayesian", "loss_function" : 'RMSE',"eval_metric":"RMSE","has_time": True}

cat_params_1 = {'depth': 6, 'learning_rate': 0.02562650627754423, 'l2_leaf_reg': 0.6658610403321601,
                'grow_policy': 'SymmetricTree', 'bagging_temperature': 4.585387000263623,"random_seed": 42,"iterations": 5000,
                "bootstrap_type":"Bayesian", "loss_function" : 'RMSE',"eval_metric":"RMSE","has_time": True}

In [ ]:
from catboost import CatBoostRegressor

import joblib
def train_and_evaluate_cat(train, test, params, boost=5000):
    test = test.drop('time_id',axis=1)
    features = [col for col in train.columns if col not in { "target", "row_id", "time_id" }]
    #features = [col for col in train.columns if col not in {"target", "row_id"}]
    X = train[features] 
    y = train['target']
    # Create out of folds array
    oof_predictions = np.zeros(train.shape[0])
    
    # Create test array to store predictions
    test_predictions = np.zeros(test.shape[0])
    
    # Create a KFold object
    num_folds = 5
    # Group Fold
    group_fold = GroupKFold( n_splits = num_folds )
    
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(group_fold.split(X, y, groups = train['time_id'])):
        print(f'Training fold {fold + 1}')
        x_train, x_val = X.iloc[trn_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_pool = Pool(x_train, y_train,weight=train_weights)
        val_pool = Pool(x_val, y_val,weight=val_weights)
        model = CatBoostRegressor(**params)
        
        model.fit(train_pool,use_best_model =True, eval_set = val_pool, 
                  verbose = 250,early_stopping_rounds=20,plot =True)
        oof_predictions[val_ind] = model.predict(x_val[features])
        
        # Predict the test set
        test_predictions += model.predict(test[features]) / num_folds
    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    print(30*'-')
    # Return test predictions
    return test_predictions

In [ ]:
CAT_TRAIN = False
if CAT_TRAIN:
    predictions_cat_1 = train_and_evaluate_cat( df_training, df_testing, cat_params_1 )

# LGB Training 

In [ ]:
new_params = {'learning_rate': 0.04617790260722144, 'lambda_l1': 0.7812638405539516, 
              'lambda_l2': 0.10048137762832532, 'num_leaves': 305, 'feature_fraction': 0.7620458604190743,
              'bagging_fraction': 0.8666298803788923, 'bagging_freq': 2, 'min_data_in_leaf': 574, 'max_depth': 11,'seed': 1111,
              'objective': 'rmse','boosting': 'gbdt','verbosity': -1,'n_jobs': -1,} # 0.2288

new_params_1 = {'learning_rate': 0.12100470091570181, 'lambda_l1': 0.9484882120442775, 
                'lambda_l2': 0.18592387603709012, 'num_leaves': 747, 'feature_fraction': 0.5464453484366312, 
                'bagging_fraction': 0.8852154214540782, 'bagging_freq': 2, 'min_data_in_leaf': 533, 'max_depth': 9,
               'seed': 1111,'objective': 'rmse','boosting': 'gbdt','verbosity': -1,'n_jobs': -1} # 0.22913

new_params_2 = {'learning_rate': 0.0798333323013617, 'lambda_l1': 0.931510376585135, 
                'lambda_l2': 0.9836497055219714, 'num_leaves': 750, 'feature_fraction': 0.5975156297650424, 
                'bagging_fraction': 0.8211686012186546, 'bagging_freq': 1, 'min_data_in_leaf': 494, 'max_depth': 12,
                 'seed': 1111,'objective': 'rmse','boosting': 'gbdt','verbosity': -1,'n_jobs': -1} # 0.2290
new_params_3 = {'learning_rate': 0.035210179568149945, 'lambda_l1': 0.2785515724790116, 'lambda_l2': 0.1344375389299167, 
                'num_leaves': 420, 'feature_fraction': 0.7424685714197569, 
                'bagging_fraction': 0.7501140603166803, 'bagging_freq': 1, 'min_data_in_leaf': 639, 'max_depth': 11,
               'seed': 1111,'objective': 'rmse','boosting': 'gbdt','verbosity': -1,'n_jobs': -1} # 0.2287

In [ ]:
import joblib
def train_and_evaluate_lgb(train, test, params, boost=5000):
    test['stock_id'] = test['stock_id'].astype('category')
    test = test.drop('time_id',axis=1)
    features = [col for col in train.columns if col not in { "target", "row_id", "time_id" }]
    X = train[features]
    X['stock_id'] = X['stock_id'].astype('category')
    y = train['target']
    # Create out of folds array
    oof_predictions = np.zeros(train.shape[0])
    
    # Create test array to store predictions
    test_predictions = np.zeros(test.shape[0])
    
    # Create a KFold object
    num_folds = 5
    # Group Fold
    group_fold = GroupKFold( n_splits = num_folds )
    
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(group_fold.split(X, y, groups = train['time_id'])):
        print(f'Training fold {fold + 1}')
        x_train, x_val = X.iloc[trn_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train, y_train, weight = train_weights)
        val_dataset = lgb.Dataset(x_val, y_val, weight = val_weights)
        model = lgb.train(params = params,
                          num_boost_round=boost,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          verbose_eval = 250,
                          early_stopping_rounds=20,
                          feval = feval_rmspe)
        
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val[features])
        
        # Predict the test set
        test_predictions += model.predict(test[features]) / num_folds
        
        # Save model
        joblib.dump(model, f'model_fold{fold}.pkl')
        print(f'Model {fold + 1} Saved...')
        
    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    print(30*'-')
    lgb.plot_importance(model,max_num_features=20)
    
    # Return test predictions
    return test_predictions

In [ ]:
LGB_TRAIN = True
if LGB_TRAIN:
    predictions_lgb_1 = train_and_evaluate_lgb( df_training, df_testing, new_params )
    predictions_lgb_2 = train_and_evaluate_lgb( df_training, df_testing, new_params_1 )
    predictions_lgb_3 = train_and_evaluate_lgb( df_training, df_testing, new_params_2 )
    predictions_lgb_4 = train_and_evaluate_lgb( df_training, df_testing, new_params_3 )

In [ ]:
# LGBM and CatBoost Ensemble
df_testing['target'] = ((predictions_lgb_1 * 0.35) + (predictions_lgb_2 * 0.15) + (predictions_lgb_3 * 0.15) + (predictions_lgb_4 * 0.35))# * 0.95 + (predictions_cat_1) * 0.05
df_testing[['row_id', 'target']].to_csv('submission.csv',index = False)

# Submission CSV

In [ ]:
pd.read_csv("submission.csv")